In [ ]:
#Soooo you can test for non f1 scores with some changes to the functions cuz i dont want to make enums
%pip install scikit-learn

In [28]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn import metrics
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from scipy.sparse import csr_matrix
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [8]:
tweets = pd.read_pickle("py_df.pkl")

In [9]:

def Tree_Test(my_df, maximum_features = None, random = None, depth = 6, test_pct = .35, n_gram_min = 1, n_gram_max = 1):
    tweets = my_df
    tweets = tweets[tweets['hate'].isin(['0', '1'])].copy()

    labels = tweets['hate'].astype(int)

    tweets['joined_tokens'] = tweets['noStop'].apply(lambda toks: " ".join(toks))

    vectorizer = CountVectorizer(max_features=maximum_features, ngram_range=(n_gram_min,n_gram_max))
    X = vectorizer.fit_transform(tweets['joined_tokens'])
    feature_names = vectorizer.get_feature_names_out()

    # Train test split
    X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=test_pct, random_state=random)

    # max depth 6
    dt6 = tree.DecisionTreeClassifier(max_depth=depth, random_state=random)
    dt6 = dt6.fit(X_train, y_train)

    predictions_binary = dt6.predict(X_test)

    f1 = metrics.f1_score(y_test, predictions_binary, zero_division=0)

    return f1

In [10]:

def Bayes_test(my_df, feature_maximum = None, random = None, test_pct = .35, n_gram_min = 1, n_gram_max = 1):

    tweets = my_df
    tweets = tweets[tweets['hate'].isin(['0', '1'])].copy()
    labels = tweets['hate'].astype(int)

    tweets['joined_tokens'] = tweets['noStop'].apply(lambda toks: " ".join(toks))

    vectorizer = CountVectorizer(max_features=feature_maximum, ngram_range=(n_gram_min,n_gram_max))
    X = vectorizer.fit_transform(tweets['joined_tokens'])
    feature_names = vectorizer.get_feature_names_out()

    X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=test_pct, random_state=random)

    clf = MultinomialNB()
    clf.fit(X_train, y_train)

    predictions = clf.predict(X_test)
    f1 = metrics.f1_score(y_test, predictions, zero_division=0)
    return f1





In [11]:

def Adaboost_Bayes_test(my_df, feature_maximum = None, random = None, test_pct = .35, n_gram_min = 1, n_gram_max = 1, estimators = 50, learn_rate = 1.0):

    tweets = my_df
    tweets = tweets[tweets['hate'].isin(['0', '1'])].copy()
    labels = tweets['hate'].astype(int)

    tweets['joined_tokens'] = tweets['noStop'].apply(lambda toks: " ".join(toks))

    vectorizer = CountVectorizer(max_features=feature_maximum, ngram_range=(n_gram_min,n_gram_max))
    X = vectorizer.fit_transform(tweets['joined_tokens'])
    feature_names = vectorizer.get_feature_names_out()

    X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=test_pct, random_state=random)

    clf = AdaBoostClassifier(estimator=MultinomialNB(), n_estimators=estimators,learning_rate=learn_rate,random_state=random)
    clf.fit(X_train, y_train)

    predictions = clf.predict(X_test)
    f1 = metrics.f1_score(y_test, predictions, zero_division=0)
    return f1


In [12]:
def forest_test(my_df,feature_maximum = None, random = None, test_pct = .35, n_gram_min = 1, n_gram_max = 1, estimators = 100, tree_height = None, min_split = 2, bagging = True, paralleled = -1):
    tweets = my_df
    tweets = tweets[tweets['hate'].isin(['0', '1'])].copy()
    labels = tweets['hate'].astype(int)

    tweets['joined_tokens'] = tweets['noStop'].apply(lambda toks: " ".join(toks))

    vectorizer = CountVectorizer(max_features=feature_maximum, ngram_range=(n_gram_min,n_gram_max))
    X = vectorizer.fit_transform(tweets['joined_tokens'])
    feature_names = vectorizer.get_feature_names_out()

    X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=test_pct, random_state=random)

    #May want to change estimator type, I just left it as Gini
    #There are so many freaking parameters for this function. Do some research before testing
    clf = RandomForestClassifier(n_estimators= estimators, max_depth=tree_height, min_samples_split=min_split, bootstrap=bagging, n_jobs=paralleled)
    clf.fit(X_train, y_train)

    predictions = clf.predict(X_test)
    f1 = metrics.f1_score(y_test, predictions, zero_division=0)
    return f1

In [19]:
def CatBoost_test(my_df, feature_maximum=None, random=None, test_pct=0.35, 
                  n_gram_min=1, n_gram_max=1, iterations=100, depth=6, 
                  learning_rate=0.1):
    tweets = my_df
    tweets = tweets[tweets['hate'].isin(['0', '1'])].copy()
    labels = tweets['hate'].astype(int)

    tweets['joined_tokens'] = tweets['noStop'].apply(lambda toks: " ".join(toks))

    vectorizer = CountVectorizer(max_features=feature_maximum, ngram_range=(n_gram_min, n_gram_max))
    X = vectorizer.fit_transform(tweets['joined_tokens'])

    X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=test_pct, random_state=random)


    clf = CatBoostClassifier(iterations=iterations, depth=depth, learning_rate=learning_rate, random_seed=random, verbose=False)

    clf.fit(X_train, y_train)
    predictions = clf.predict(X_test)
    f1 = metrics.f1_score(y_test, predictions, zero_division=0)
    return f1

In [13]:
Tree_Test(tweets,5000, 50, 10, test_pct= .35, n_gram_min=1, n_gram_max=2)

0.24330077270739175

In [ ]:
Bayes_test(tweets, n_gram_max=2)

0.5431937172774869

In [ ]:
Adaboost_Bayes_test(tweets, learn_rate=0.5, random=None, estimators=200, n_gram_max=2)

0.5659952606635071

In [ ]:
forest_test(tweets, feature_maximum=7500)

0.5999403281949279

In [20]:
CatBoost_test(tweets, feature_maximum=5000, random=None, test_pct=0.25)

NameError: name 'CatBoostClassifier' is not defined